# Momentum Strategy

### Picking a stock universe - e.g. Dow Jones Industrial Average 

### Performance evaluation: checking the performance over every single stock over the last 12 month
### skip the most recent month => buy winners & short losers 

#### Highest quantile = winners 
#### lowest quintile = losers 

##### Calculate the strategy return and compare return to the benchmark -> Dow Jones Performance

In [5]:
import yfinance as yf
import pandas_datareader.data as reader
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta

In [11]:
# Get ticker symbols for the stocks contained in DJI

wiki_url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
tables   = pd.read_html(wiki_url)

# find the table with “Symbol” & “Company”
for tbl in tables:
    if {"Symbol","Company"}.issubset(tbl.columns):
        dji_components = tbl
        break

table = dji_components
tickers = table.Symbol.tolist()
tickers

['MMM',
 'AXP',
 'AMGN',
 'AMZN',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DIS',
 'GS',
 'HD',
 'HON',
 'IBM',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'NVDA',
 'PG',
 'CRM',
 'SHW',
 'TRV',
 'UNH',
 'VZ',
 'V',
 'WMT']

In [13]:
# Get prices for the DJI componets
start = dt.datetime(2018,1,31)
end = dt.datetime(2020,1,31)

df = yf.download(tickers, start, end, auto_adjust = False)['Adj Close']
df

[*********************100%***********************]  30 of 30 completed


Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,39.293301,148.054565,72.544502,89.722427,337.712067,137.699844,112.920532,33.149426,90.738747,103.949203,...,87.690552,62.288891,6.077598,70.879700,130.082962,127.049706,211.059097,117.970703,36.176826,31.400476
2018-02-01,39.375439,147.664581,69.500000,90.264008,340.161255,137.243011,111.760689,33.277096,90.898003,105.690147,...,86.998322,61.768444,5.946550,70.477455,128.529816,127.117516,209.668533,119.385605,36.330704,31.082348
2018-02-02,37.666935,148.818481,71.497498,87.267235,332.508820,133.224884,109.817711,32.662628,85.838043,103.977890,...,84.709396,61.375816,5.773964,69.163940,125.723038,125.380287,206.691315,114.818001,35.447533,30.776016
2018-02-05,36.725849,139.062271,69.500000,83.051903,313.420319,127.802437,106.714905,30.946899,81.523727,100.151672,...,81.220612,58.791855,5.283900,66.545151,122.635536,119.931168,196.119705,110.411766,33.788227,29.482878
2018-02-06,38.260681,140.574234,72.141998,85.010635,324.884796,132.311264,108.618233,32.056137,84.824646,101.557808,...,84.294075,59.549698,5.577642,67.628761,126.306168,119.363358,200.719162,113.925331,34.009018,29.721472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-24,76.963326,190.414429,93.082001,125.538094,321.139313,125.143089,180.528122,41.298122,87.506561,137.783051,...,157.253754,95.209564,6.234852,109.161293,188.836624,119.597443,268.995819,197.225662,43.991627,35.187347
2020-01-27,74.700211,188.186050,91.417000,121.384766,314.727478,120.997787,179.140289,40.131470,86.364334,133.671600,...,154.623962,93.539230,5.978965,109.641075,185.481018,118.665718,260.985016,194.041199,43.721596,35.645771
2020-01-28,76.813431,189.781357,92.662498,123.066521,314.687714,121.898178,181.261688,40.385094,86.935448,136.101089,...,157.653961,93.697861,6.172373,109.937660,187.633423,119.251373,262.620209,195.157181,44.298126,35.873447


In [16]:
# calculating the monthly returns
mtl_ret = df.pct_change().resample('M').agg(lambda x:(x+1).prod() - 1)
mtl_ret

Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-02-28,0.068185,-0.004788,0.042429,-0.019014,0.027169,-0.050068,0.020543,0.077997,-0.098252,-0.050704,...,-0.008415,-0.017444,-0.014848,-0.090572,-0.035153,-0.072839,-0.044852,-0.008661,-0.117070,-0.155628
2018-03-31,-0.058051,-0.072319,-0.043049,-0.043380,-0.094779,-0.046886,0.000430,-0.042207,0.018942,-0.026367,...,-0.026661,-0.005804,-0.043017,0.009679,-0.023557,0.004199,-0.050634,-0.027005,0.001676,-0.005661
2018-04-30,-0.015020,0.023463,0.082075,0.062615,0.017323,-0.015560,0.040327,0.040943,0.097071,-0.001095,...,0.024652,0.029350,-0.028887,-0.079095,-0.062379,-0.052283,0.104673,0.060692,0.044980,-0.005732
2018-05-31,0.135124,0.037404,0.040539,-0.004557,0.061029,0.052300,0.068931,-0.035674,0.002189,-0.008572,...,0.061467,0.049861,0.122036,0.011474,0.033857,-0.023404,0.021616,0.031920,-0.034042,-0.061045
2018-06-30,-0.009418,0.027669,0.043065,-0.003052,-0.047279,-0.106905,0.054666,0.007493,0.017136,0.053684,...,-0.002327,0.112850,-0.060629,0.066831,0.074674,-0.042360,0.019452,0.013234,0.055381,0.037679
2018-07-31,0.027984,0.064792,0.045676,0.019156,0.061965,0.066424,0.005499,-0.009537,-0.001265,0.092194,...,0.075753,-0.034764,0.033601,0.045530,0.081360,0.063757,0.032119,0.032390,0.038336,0.041798
2018-08-31,0.200422,0.023505,0.132364,0.064912,-0.033164,-0.034423,0.113234,0.129581,-0.052857,-0.013561,...,0.062993,0.071414,0.146915,0.025593,0.035725,0.011219,0.060185,0.075843,0.052866,0.080542
2018-09-30,-0.008303,0.037436,-0.004824,0.004812,0.084921,0.098236,0.041590,0.018422,0.032247,0.043921,...,0.018161,0.030657,0.001211,0.003376,-0.000813,-0.008603,-0.005671,0.021785,-0.018025,-0.020342


In [17]:
#calculate the return s of the last 11 month
import numpy as np

past_11 = (mtl_ret+1).rolling(11).apply(np.prod) - 1
past_11

Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MSFT,NKE,NVDA,PG,SHW,TRV,UNH,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# define formation day
formation = dt.datetime(2019,12,31)

end_measurement = formation - relativedelta(month=1)

In [21]:
formation

datetime.datetime(2019, 12, 31, 0, 0)

In [24]:
end_measurement

datetime.datetime(2019, 1, 31, 0, 0)

In [32]:
#loc functiom to find the end_measurement in that dataframe 
ret_12 = past_11.loc[end_measurement]
ret_12

Ticker
AAPL   -0.055396
AMGN    0.040230
AMZN    0.136388
AXP     0.069161
BA      0.080194
CAT    -0.118018
CRM     0.307269
CSCO    0.088455
CVX     0.053428
DIS     0.098182
GS     -0.238673
HD      0.029667
HON     0.007208
IBM    -0.107167
JNJ     0.045734
JPM    -0.080711
KO      0.152121
MCD     0.154781
MMM    -0.132062
MRK     0.414817
MSFT    0.127664
NKE     0.235342
NVDA   -0.404704
PG      0.272267
SHW     0.056284
TRV    -0.075655
UNH     0.210841
V       0.103574
VZ      0.206939
WMT     0.089929
Name: 2019-01-31 00:00:00, dtype: float64

In [37]:
#transform series into dataframe 
ret_12 = ret_12.reset_index()
ret_12

,Ticker,2019-01-31 00:00:00
0,AAPL,-0.055396
1,AMGN,0.040230
2,AMZN,0.136388
3,AXP,0.069161
4,BA,0.080194
5,CAT,-0.118018
6,CRM,0.307269
7,CSCO,0.088455
8,CVX,0.053428
9,DIS,0.098182


In [38]:
# use quintiles to divide the distribution into 5 equal parts to see the winners [4} and losers [0]
# 1) coerce your returns columns to numeric (so numpy.isnan will work)
ret_12.iloc[:,1:] = ret_12.iloc[:,1:].apply(pd.to_numeric, errors='coerce')

In [40]:
# 2a) If you just want **one** quintile column based on a single return series (say the 2nd column):
ret_12['quintile'] = pd.qcut(
    ret_12.iloc[:,1],  # ← note: a Series, not a DataFrame
    5,
    labels=False
)
ret_12

,Ticker,2019-01-31 00:00:00,quintile
0,AAPL,-0.055396,1
1,AMGN,0.040230,1
2,AMZN,0.136388,3
3,AXP,0.069161,2
4,BA,0.080194,2
5,CAT,-0.118018,0
6,CRM,0.307269,4
7,CSCO,0.088455,2
8,CVX,0.053428,2
9,DIS,0.098182,3


In [42]:
winners = ret_12[ret_12.quintile == 4]
losers = ret_12[ret_12.quintile == 0]

In [43]:
winners

,Ticker,2019-01-31 00:00:00,quintile
6,CRM,0.307269,4
19,MRK,0.414817,4
21,NKE,0.235342,4
23,PG,0.272267,4
26,UNH,0.210841,4
28,VZ,0.206939,4


In [44]:
losers

,Ticker,2019-01-31 00:00:00,quintile
5,CAT,-0.118018,0
10,GS,-0.238673,0
13,IBM,-0.107167,0
15,JPM,-0.080711,0
18,MMM,-0.132062,0
22,NVDA,-0.404704,0


In [53]:
winner_ret = mtl_ret.loc[formation + relativedelta(month=1), df.columns.isin(winners.Ticker)]
loser_ret = mtl_ret.loc[formation + relativedelta(month=1), df.columns.isin(losers.Ticker)]

In [47]:
winner_ret

Ticker
CRM    0.109513
MRK   -0.025913
NKE    0.104397
PG     0.057800
UNH    0.084618
VZ    -0.010412
Name: 2019-01-31 00:00:00, dtype: float64

In [48]:
loser_ret

Ticker
CAT     0.054668
GS      0.185333
IBM     0.182546
JPM     0.068844
MMM     0.051223
NVDA    0.076779
Name: 2019-01-31 00:00:00, dtype: float64

In [57]:
#equal weighting the portfolio
momentum = winner_ret.mean() - loser_ret.mean()

In [58]:
momentum

-0.04989807332815863

In [60]:
#check how DJI performed in Jan to compare
DJI = yf.download('^DJI', start, end, auto_adjust = False)['Adj Close'].pct_change().resample('M').agg(lambda x: (x+1).prod() -1 )
DJI

[*********************100%***********************]  1 of 1 completed


Ticker,^DJI
Date,
2018-01-31,0.000000
2018-02-28,-0.042838
2018-03-31,-0.037000
2018-04-30,0.002491
2018-05-31,0.010458
2018-06-30,-0.005915
2018-07-31,0.047125
2018-08-31,0.021626
2018-09-30,0.019006


### One month after formation (i.e. in February 2019), you pulled each group’s month‑end return:

### Winners’ Feb‑2019 returns:
### CRM +10.95%, MRK −2.59%, NKE +10.44%, PG +5.78%, UNH +8.46%, VZ −1.04%

### Losers’ Feb‑2019 returns:
### CAT +5.47%, GS +18.53%, IBM +18.25%, JPM +6.88%, MMM +5.12%, NVDA +7.68% 

## losing about -4.9% per month
## benchmark - + 3.66% per month

### Negative strategy return (–4.99%) means that in this single formation → holding window the “classic” momentum signal failed: the bottom‐past‑11‑month group actually outperformed the top group.

### DJIA benchmark (+3.67%) was positive—so a simple “buy index” would have beaten your long‑winners/short‑losers trade by ~8.7 percentage points that month.